In [2]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

In [5]:
# 1. CSV 파일 불러오기
df = pd.read_csv("../jeju_final_2.csv", encoding="utf-8-sig")

print(df.head())

       출발행정코드      출발도   출발시  출발동             출발 주소      도착행정코드      도착도  \
0  5013059000  제주특별자치도  서귀포시  대륜동  제주특별자치도 서귀포시 대륜동  5013054000  제주특별자치도   
1  5013054000  제주특별자치도  서귀포시  천지동  제주특별자치도 서귀포시 천지동  5013057000  제주특별자치도   
2  5013057000  제주특별자치도  서귀포시  동홍동  제주특별자치도 서귀포시 동홍동  5013054000  제주특별자치도   
3  5013054000  제주특별자치도  서귀포시  천지동  제주특별자치도 서귀포시 천지동  5013059000  제주특별자치도   
4  5011062000  제주특별자치도   제주시  봉개동   제주특별자치도 제주시 봉개동  5011059000  제주특별자치도   

    도착시  도착동             도착 주소  
0  서귀포시  천지동  제주특별자치도 서귀포시 천지동  
1  서귀포시  동홍동  제주특별자치도 서귀포시 동홍동  
2  서귀포시  천지동  제주특별자치도 서귀포시 천지동  
3  서귀포시  대륜동  제주특별자치도 서귀포시 대륜동  
4   제주시  건입동   제주특별자치도 제주시 건입동  


In [11]:
# 2. Kakao REST API 키
api_key = "551e808d7c904940ef5197c202e1725c"  # 예: "69e904e85329cddd6795bf2cfe6c576a"

In [10]:
# 3. 주소 → 위도/경도 요청 함수
def get_lat_lng(address, api_key):
    try:
        url = "https://dapi.kakao.com/v2/local/search/address.json"
        headers = {"Authorization": f"KakaoAK {api_key}"}
        params = {"query": address}
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            result = response.json()
            if result['documents']:
                location = result['documents'][0]['address']
                return location['y'], location['x']  # 위도(y), 경도(x)
        return None, None
    except:
        return None, None

In [17]:
# 4. 전체 주소 중, 30,000번째 이후만 추출
addresses1 = df["출발 주소"].tolist()  # 인덱스 

# 5. 병렬 처리
lat_list, lng_list = [], []

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(get_lat_lng, addr, api_key): addr for addr in addresses1}

    for future in tqdm(as_completed(futures), total=len(futures)):
        lat, lng = future.result()
        lat_list.append(lat)
        lng_list.append(lng)

100%|█████████████████████████████████████████████████████████████████████████████| 4992/4992 [00:33<00:00, 147.14it/s]


In [18]:
# 6. 부분 결과 DataFrame 생성
df["위도"] = lat_list
df["경도"] = lng_list

# 7. 저장
df.to_csv("제주도_출발_주소_좌표.csv", index=False, encoding="utf-8-sig")
print("✔️ 저장 완료: 당신의 파일_좌표추가_전체.csv")

✔️ 저장 완료: 당신의 파일_좌표추가_전체.csv


In [15]:
# 4. 전체 주소 중, 30,000번째 이후만 추출
addresses2 = df["도착 주소"].tolist()  # 인덱스 

# 5. 병렬 처리
lat_list, lng_list = [], []

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(get_lat_lng, addr, api_key): addr for addr in addresses2}

    for future in tqdm(as_completed(futures), total=len(futures)):
        lat, lng = future.result()
        lat_list.append(lat)
        lng_list.append(lng)

100%|█████████████████████████████████████████████████████████████████████████████| 4992/4992 [00:33<00:00, 148.48it/s]


In [ ]:
# 6. 부분 결과 DataFrame 생성
df["위도"] = lat_list
df["경도"] = lng_list

# 7. 저장
df.to_csv("제주도_도착_주소_좌표.csv", index=False, encoding="utf-8-sig")
print("✔️ 저장 완료: 당신의 파일_좌표추가_전체.csv")

In [ ]:
## csv 병합
# 1. 두 CSV 파일 불러오기
df1 = pd.read_csv("제주도_출발_주소_좌표.csv", encoding="utf-8-sig")
df2 = pd.read_csv("제주도_도착_주소_좌표.csv", encoding="utf-8-sig")

# 2. 병합
df_merged = pd.concat([df1, df2], ignore_index=True)
df_merged = df_merged.drop_duplicates()

# 3. 저장
df_merged.to_csv("제주도_전체_주소_좌표.csv", index=False, encoding="utf-8-sig")
print("✔️ 최종 저장 완료: 서울_충전소_좌표추가_전체.csv")